# **Models for my Final Project - "IntelliFleetManager"**

In [ ]:
# For organizing the data
import pandas as pd
import numpy as np
from keras.utils import np_utils

# For model 1 + 2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
# For model 3
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
# Load the dataset
data = pd.read_csv('obd2data.csv')

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------------------------#

# **Model 1 : Predicts if there is an issue or not**

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------------------------#

In [ ]:
# Split the data into training and testing sets
X1 = data.drop(['issues','trouble_codes','time','vehicle_id','id','ip'], axis=1).values
y1 = data['issues'].values
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [ ]:
print(X1)

[[ 8.000e+01  3.330e-01  1.009e+03 ...  0.000e+00  2.510e-01  5.690e-01]
 [ 8.000e+01  3.250e-01  1.003e+03 ...  0.000e+00  2.510e-01  5.650e-01]
 [ 8.000e+01  3.290e-01  9.950e+02 ...  0.000e+00  2.510e-01  5.730e-01]
 ...
 [-1.000e+00 -1.000e+00 -1.000e+00 ...  0.000e+00 -1.000e+00 -1.000e+00]
 [-1.000e+00 -1.000e+00 -1.000e+00 ...  0.000e+00 -1.000e+00 -1.000e+00]
 [-1.000e+00 -1.000e+00 -1.000e+00 ...  0.000e+00 -1.000e+00 -1.000e+00]]


In [ ]:
print(y1)

[0 0 0 ... 0 0 0]


In [ ]:
# Build a neural network model
model1 = tf.keras.Sequential([
tf.keras.layers.Dense(64, input_shape=(X_train1.shape[1],), activation='relu'),
tf.keras.layers.Dense(32, activation='relu'),
tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model1.fit(X_train1, y_train1, epochs=120, batch_size=32, verbose=1)

Epoch 1/120
1184/1184 [==============================] - 3s 2ms/step - loss: 1.8900 - accuracy: 0.7617
Epoch 2/120
1184/1184 [==============================] - 3s 2ms/step - loss: 0.8153 - accuracy: 0.8015
Epoch 3/120
1184/1184 [==============================] - 4s 3ms/step - loss: 0.5809 - accuracy: 0.8328
Epoch 4/120
1184/1184 [==============================] - 2s 2ms/step - loss: 0.5881 - accuracy: 0.8422
Epoch 5/120
1184/1184 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.8495
Epoch 6/120
1184/1184 [==============================] - 2s 2ms/step - loss: 0.4964 - accuracy: 0.8618
Epoch 7/120
1184/1184 [==============================] - 3s 3ms/step - loss: 0.5114 - accuracy: 0.8628
Epoch 8/120
1184/1184 [==============================] - 4s 3ms/step - loss: 0.4154 - accuracy: 0.8695
Epoch 9/120
1184/1184 [==============================] - 3s 2ms/step - loss: 0.3785 - accuracy: 0.8799
Epoch 10/120
1184/1184 [==============================] - 2s 2ms/step - l

In [ ]:
# Evaluate the performance of the model
loss1, accuracy1 = model1.evaluate(X_test1, y_test1)
print('Accuracy:', accuracy1)

296/296 [==============================] - 25s 1ms/step - loss: 0.1471 - accuracy: 0.9418
Accuracy: 0.9417978525161743


In [ ]:
y_pred = model1.predict(X_test1)
y_pred = np.round(y_pred).astype(int)
print(y_pred)

296/296 [==============================] - 0s 1ms/step
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------------------------#

# **Model 2 : Predicts which issue**

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------------------------#

In [ ]:
# Filter the dataset to include only rows where "issues" equals 1
X2 = data[data['issues'] == 1].copy()

# Make a y2 data based on X2
temp_y2 = X2['trouble_codes'].values

one_hot_encoder = OneHotEncoder(sparse=False)
temp_y2 = one_hot_encoder.fit_transform(temp_y2.reshape(-1, 1))
y2 = np.argmax(temp_y2, axis=1)  # Convert one-hot encoded labels to integer labels

# Drop the noneed columns from X2
X2.drop(['issues', 'trouble_codes', 'time', 'vehicle_id', 'id', 'ip'], axis=1, inplace=True)

In [ ]:
# Split the data into training and testing sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42) 

# Make num_categories for the number of the trouble codes we have in our y_train2
num_categories = len(np.unique(y_train2))

In [ ]:
print(y_train2.shape)

(9540,)


In [ ]:
print(X_train2.shape[1])

10


In [ ]:
print(num_categories)

13


In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape=(X_train2.shape[1],), activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_categories, activation='softmax')
])


In [ ]:
model2.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 64)                704       
                                                                 
 dense_31 (Dense)            (None, 32)                2080      
                                                                 
 dense_32 (Dense)            (None, 13)                429       
                                                                 
Total params: 3,213
Trainable params: 3,213
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model2.fit(X_train2, y_train2, epochs=120, batch_size=32, verbose=1)

Epoch 1/120
299/299 [==============================] - 1s 2ms/step - loss: 10.0209 - accuracy: 0.4993
Epoch 2/120
299/299 [==============================] - 1s 3ms/step - loss: 1.7582 - accuracy: 0.5415
Epoch 3/120
299/299 [==============================] - 1s 3ms/step - loss: 1.9049 - accuracy: 0.5522
Epoch 4/120
299/299 [==============================] - 1s 3ms/step - loss: 1.3737 - accuracy: 0.5543
Epoch 5/120
299/299 [==============================] - 1s 3ms/step - loss: 1.4761 - accuracy: 0.5581
Epoch 6/120
299/299 [==============================] - 1s 3ms/step - loss: 1.3184 - accuracy: 0.5805
Epoch 7/120
299/299 [==============================] - 1s 2ms/step - loss: 1.1995 - accuracy: 0.5926
Epoch 8/120
299/299 [==============================] - 1s 2ms/step - loss: 1.1058 - accuracy: 0.6034
Epoch 9/120
299/299 [==============================] - 1s 2ms/step - loss: 1.1217 - accuracy: 0.5927
Epoch 10/120
299/299 [==============================] - 1s 2ms/step - loss: 1.1898 - accur

In [ ]:
# Evaluate the performance of the model
loss2, accuracy2 = model2.evaluate(X_test2, y_test2)
print('Accuracy:', accuracy2)

75/75 [==============================] - 0s 2ms/step - loss: 0.3673 - accuracy: 0.8780
Accuracy: 0.8779874444007874


In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------------------------#

# **Model 3 : Predicts which issue will appear next**

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------------------------#

In [ ]:

# Filter for rows where "issues" == 1
filtered_data = data[data["issues"] == 1].copy()

# Create a pivot table to get all unique trouble codes for each car ID
pivot_table = filtered_data.pivot_table(index=["vehicle_id"], values=["trouble_codes"], aggfunc=lambda x: tuple(np.unique(x)))

# Convert the pivot table to a NumPy array
trouble_data = pivot_table.to_numpy()

In [ ]:
# Print the list of arrays
print(trouble_data)

[[('P0133',)]
 [('P0133',)]
 [('P0133',)]
 [('C0300',)]
 [('C0300',)]
 [('P0078B0004P3000', 'P0078U1004P3000', 'P0079C1004P3000', 'P0079P1004P3000', 'P0079P2004P3000', 'P007EP2036P18D0', 'P007EP2036P18E0', 'P007EP2036P18F0', 'P007FP2036P18D0', 'P007FP2036P18E0', 'P007FP2036P18F0')]]


In [ ]:
# Convert the dataset into a pandas DataFrame
dataset = pd.DataFrame(trouble_data)
dataset

,0
0,"(P0133,)"
1,"(P0133,)"
2,"(P0133,)"
3,"(C0300,)"
4,"(C0300,)"
5,"(P0078B0004P3000, P0078U1004P3000, P0079C1004P..."


In [ ]:
# Transform the dataset into a one-hot encoded matrix
onehot = pd.get_dummies(dataset.apply(pd.Series).stack()).sum(level=0)

<ipython-input-152-cad5a7d33352>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  onehot = pd.get_dummies(dataset.apply(pd.Series).stack()).sum(level=0)


In [ ]:
# Generate frequent itemsets using the Apriori algorithm
frequent_itemsets = apriori(onehot, min_support=0.3, use_colnames=True)

In [ ]:
# Generate association rules from frequent itemsets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [ ]:
# Print the association rules
print(rules)

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []
